# Life Cycle Analysis (LCA) Automation Tool
*This project is a Python tool developed to analyze, compare, and visualize the environmental impacts of various products throughout their life cycles.*

**Author:** İlker Mehmet ÇEVİK
**Last Updated:** June 12, 2025

Life Cycle Analysis (LCA) is a methodology for assessing the environmental impacts of a product across its entire life span, from raw material extraction ("cradle") to its final disposal ("grave"). It quantifies key metrics like carbon footprint, water usage, and energy consumption at every stage, providing a complete and holistic picture of a product's true environmental footprint.

The goal of this tool is to automate the LCA process for construction materials, enabling a direct and data-driven comparison of their environmental performance. By calculating and visualizing key impact indicators, it empowers engineers and architects to quickly assess the sustainability of different design choices and select materials with a lower environmental footprint.

This document serves a dual purpose: it is both a technical guide and a practical showcase. As a guide, it provides the necessary instructions and API reference to use and extend the tool. As a showcase, it presents an end-to-end analysis, demonstrating the tool's capabilities in generating actionable insights and validating its real-world application.

## 2. Setup and Installation
This section describes the necessary steps to run the project and explains the project's file structure.

### 2.1. Required Libraries
The project requires the following libraries: `pandas`, `numpy`, `matplotlib`, `seaborn`. To install these dependencies, run the following command in your terminal from the project's root directory:

```bash
pip install -r requirements.txt

### 2.2. Project File Structure
The project follows a standard Python project structure:

```
final_project/
├── data/
│   └── raw/
│       ├── impact_factors.json
│       └── sample_data.csv
├── notebooks/
│   ├── documentation.ipynb
│   └── lca_analysis_example.ipynb
├── outputs/                      <-- Generated automatically upon execution
│   ├── data/
│   └── figures/
├── src/
│   ├── __init__.py
│   ├── calculations.py
│   ├── data_input.py
│   ├── utils.py
│   └── visualization.py
├── tests/
│   ├── conftest.py
│   ├── test_calculations.py
│   ├── test_data_input.py
│   └── test_visualization.py
├── pytest.ini
└── requirements.txt
```

## 3. Quick Start: Basic Analysis Workflow
The following code blocks demonstrate the entire process in 3 steps, from loading data to creating a basic visualization.

In [ ]:
import pandas as pd
from src.data_input import DataInput
from src.calculations import LCACalculator
from src.visualization import LCAVisualizer

# Required settings for rendering in notebook
import matplotlib
%matplotlib inline

**Step 1: Data Loading**

In [ ]:
data_input = DataInput()
product_data = data_input.read_data('data/raw/sample_data.csv')
impact_factors = data_input.read_impact_factors('data/raw/impact_factors.json')
print("First 5 rows of the product data:")
product_data.head()

**Step 2: Calculation**

In [ ]:
calculator = LCACalculator(impact_factors_path='data/raw/impact_factors.json')
total_impacts = calculator.calculate_total_impacts(calculator.calculate_impacts(product_data))
print("Total environmental impacts by product:")
total_impacts

**Step 3: Visualization**

In [ ]:
visualizer = LCAVisualizer()
fig = visualizer.plot_product_comparison(calculator.calculate_impacts(product_data), ['P001', 'P002'])

## 4. Detailed Module Usage (API Reference)
This section explains the function and usage of each module under the `src` directory with detailed examples.

### 4.1. Data Input (`src/data_input.py`)

This module is responsible for all data ingestion and validation tasks. The **`DataInput`** class handles reading data from various file formats and ensures that the data conforms to the required structure before calculations begin.


**`read_data(file_path)`**

This function reads product data from a specified file path. It supports `.csv`, `.xlsx`, and `.json` formats.

In [ ]:
# Initialize the class and read the sample data
data_input = DataInput()
product_data = data_input.read_data('data/raw/sample_data.csv')

print("Data loaded successfully. Shape:", product_data.shape)
product_data.head()

**`read_impact_factors(file_path)`**

This function reads the environmental impact factors from a specified JSON file.

In [ ]:
# Read the impact factors
impact_factors = data_input.read_impact_factors('data/raw/impact_factors.json')

print("Impact factors for 'steel':")
pd.DataFrame(impact_factors['steel'])

**`validate_data(data)`**

This function checks if the input DataFrame contains all required columns and valid data types. It also validates that the end-of-life rates sum to 1. It returns `True` if the data is valid, and `False` otherwise.

In [ ]:
# Validate the loaded product data
is_valid = data_input.validate_data(product_data)

print(f"Is the product data valid? -> {is_valid}")

### 4.2. Calculations (`src/calculations.py`)

This module contains the core calculation engine of the tool, the **`LCACalculator`** class. It takes the processed data and impact factors to compute environmental impacts.

***Before running the examples below, first initialize the calculator:***

In [ ]:
# Initialize the calculator with pre-loaded impact factors
calculator = LCACalculator(impact_factors_path='data/raw/impact_factors.json')

# Perform the main impact calculation once for use in examples
impacts_df = calculator.calculate_impacts(product_data)

**`calculate_impacts(data)`**

This is the primary calculation function. It returns a new DataFrame with detailed impact results.

In [ ]:
# Display the first few rows of the detailed impact results
print("Shape of the calculated impacts DataFrame:", impacts_df.shape)
impacts_df.head()

**`calculate_total_impacts(impacts)`**

This function aggregates the detailed impact results, grouping them by product to calculate the total environmental impact for each product across its entire life cycle.

In [ ]:
# Calculate and display total impacts per product
total_impacts_df = calculator.calculate_total_impacts(impacts_df)
total_impacts_df

**`normalize_impacts(impacts)`**

This function normalizes the impact scores to a common 0-1 scale, where 1 represents the highest impact in that category.

In [ ]:
# Normalize the total impact scores for comparison
normalized_impacts = calculator.normalize_impacts(total_impacts_df)
normalized_impacts

**`compare_alternatives(impacts, product_ids)`**

This function isolates two or more products from the dataset and calculates their relative performance against each other.

In [ ]:
# Compare the environmental performance of Product 'P001' and 'P002'
comparison_df = calculator.compare_alternatives(impacts_df, ['P001', 'P002'])
comparison_df

### 4.3. Utilities (`src/utils.py`)
This module contains helper functions that can be used across the project.

**`convert_units(value, from_unit, to_unit)`**

This function converts a numerical value from a source unit to a target unit.

In [ ]:
from src.utils import convert_units

# Convert 100 kilograms to pounds
pounds = convert_units(100, from_unit='kg', to_unit='lb')

print(f"100 kg is equal to {pounds:.2f} lb")

**`save_results(data, file_path, format)`**

This function saves a pandas DataFrame to a file, supporting 'csv', 'xlsx', and 'json' formats.

In [ ]:
from src.utils import save_results
import os

os.makedirs('outputs/data', exist_ok=True)
save_results(total_impacts_df, 'outputs/data/total_impacts_summary.csv', format='csv')

print("Results saved to 'outputs/data/total_impacts_summary.csv'")

### 4.4. Visualization (`src/visualization.py`)
This module handles the creation of all plots and charts through the **`LCAVisualizer`** class.

***Before running the examples below, first initialize the visualizer:***

In [ ]:
visualizer = LCAVisualizer()

**`plot_impact_breakdown(data, impact_type, group_by)`**

Creates a pie chart showing the breakdown of a specific impact by a chosen category.

In [ ]:
# Generate a pie chart for Carbon Impact by Material Type
fig = visualizer.plot_impact_breakdown(impacts_df, 'carbon_impact', 'material_type')

**`plot_life_cycle_impacts(data, product_id)`**

Creates a set of bar charts showing impacts across the life cycle stages for a single product.

In [ ]:
# Visualize the life cycle impacts for Product 'P001'
fig = visualizer.plot_life_cycle_impacts(impacts_df, 'P001')

**`plot_product_comparison(data, product_ids)`**

Creates a radar chart to provide a head-to-head comparison of multiple products.

In [ ]:
# Generate a radar chart comparing Products 'P002' (Steel Beam) and 'P003' (Wood Beam)
fig = visualizer.plot_product_comparison(impacts_df, ['P002', 'P003'])

**`plot_end_of_life_breakdown(data, product_id)`**

Creates a stacked bar chart showing the end-of-life management strategy for a product.

In [ ]:
# Visualize the end-of-life scenario for Product 'P001'
fig = visualizer.plot_end_of_life_breakdown(impacts_df, 'P001')

**`plot_impact_correlation(data)`**

Creates a heatmap to show the correlation between different impact categories.

In [ ]:
# Generate a correlation heatmap for all impact categories
fig = visualizer.plot_impact_correlation(impacts_df)

## 5. End-to-End Analysis Scenario: Steel vs. Wood

In this section, we will use the tool to conduct a practical analysis, comparing the environmental performance of two common structural materials to answer a key question: **Which material is more sustainable?**

### 5.1. Hypothesis: Comparing a Steel Beam vs. a Wood Beam

Our hypothesis is that a modern **Engineered Wood Beam (P003)** will have a significantly lower overall environmental impact compared to a traditional **Structural Steel Beam (P002)**. We will begin by generating a radar chart to get a high-level comparison across all major impact categories.

In [ ]:
# Generate a radar chart to compare the two products
fig_scenario_1 = visualizer.plot_product_comparison(
    impacts_df, 
    ['P002', 'P003']
)

# Set the title on the figure object after it's created
fig_scenario_1.suptitle('Environmental Impact Comparison: Steel vs. Wood', fontsize=16)

### 5.2. Initial Analysis

The radar chart provides a clear visual confirmation of our hypothesis. The blue line, representing the steel beam (P002), encompasses a much larger area than the orange line for the wood beam (P003), indicating a higher impact in every category.

To quantify this, we can examine the data from our `total_impacts_df` DataFrame. The analysis reveals that the steel beam's **carbon footprint is approximately 5.6 times higher** than the engineered wood beam's. Similarly, its energy and water impacts are also significantly greater. This initial result strongly suggests that for this application, engineered wood is the more sustainable choice.

### 5.3. Drill-Down: Understanding the Steel Beam's Impact

Now that we've identified the steel beam as the higher-impact material, let's investigate *why*. We will generate a life cycle impact chart for the steel beam (P002) to see which stage contributes the most to its environmental footprint.

In [ ]:
# Generate a life cycle impact chart for the steel beam (P002)
fig_scenario_2 = visualizer.plot_life_cycle_impacts(
    impacts_df, 
    'P002'
)
# Add a main title to the figure
fig_scenario_2.suptitle('Life Cycle Impact Breakdown for Steel Beam (P002)', fontsize=16)

### 5.4. Drill-Down Analysis

The bar charts above are incredibly revealing. For the steel beam, the "Manufacturing" stage is overwhelmingly dominant across all impact types. 

A closer look at the data shows that **approximately 71% of the steel beam's total carbon footprint originates from its manufacturing process.** This highlights that while transportation and end-of-life management have an impact, the most significant opportunity for environmental improvement lies in sourcing steel produced with cleaner energy or increasing the use of recycled content.

This end-to-end analysis demonstrates how the tool can be used not only to compare products but also to pinpoint the specific "hotspots" within a product's life cycle that require the most attention for sustainability efforts.

## 6. Conclusion and Future Work

### Conclusion

This project successfully developed a modular and automated tool for performing Life Cycle Analysis. The tool provides a complete, end-to-end workflow, from data ingestion and validation to calculation and visualization, making LCA more accessible and efficient. The modular structure (`src`) and comprehensive test suite (`tests`) ensure that the tool is both reliable and easily extensible.

The analysis scenario presented in this document demonstrated the tool's practical value. By quantitatively comparing a steel beam and an engineered wood beam, we were able to validate the hypothesis that the wood beam has a significantly lower environmental footprint, particularly a carbon impact that is over five times smaller. Furthermore, the drill-down analysis successfully identified the manufacturing stage as the primary environmental "hotspot" for the steel beam, proving the tool's capability to guide targeted sustainability efforts. In conclusion, this project serves as a powerful proof-of-concept for data-driven decision-making in sustainable design.

### Future Work

While this tool is fully functional, there are numerous opportunities for future expansion and improvement. Potential next steps include:

* **Expanding the Database:**
    * Incorporate a wider range of materials (e.g., composites, advanced polymers, different alloys).
    * Integrate more comprehensive and region-specific impact factor databases (like Ecoinvent) to allow for more granular analysis based on geographical location.

* **Developing a User Interface:**
    * Create a graphical user interface (GUI), potentially as a web application using frameworks like **Streamlit** or **Flask**. This would make the tool accessible to non-programmers, allowing them to upload their own data files and generate reports interactively.

* **Increasing Analysis Depth:**
    * Add more LCA impact categories beyond carbon, water, and energy, such as **Acidification Potential (AP)**, **Eutrophication Potential (EP)**, and **Ozone Depletion Potential (ODP)**.
    * Implement uncertainty analysis (e.g., Monte Carlo simulations) to account for variability in input data and provide results as a range rather than a single point value.

* **API and Integration:**
    * Develop a REST API to allow other software to programmatically access the LCA calculation engine.
    * Explore potential integrations with architectural design software (e.g., Revit, ArchiCAD) to provide real-time environmental feedback during the design process.